In [8]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
import os

In [9]:
# check installed version
import pycaret
pycaret.__version__

'3.3.2'

In [10]:
# check installed version
import pyarrow
pyarrow.__version__

'15.0.0'

## Data Loading

In [6]:
import pandas as pd

def load_and_clean_visitor_count_data(filepath):
    """
    Load the CSV file, remove the last column, and return the cleaned DataFrame.
    
    Parameters:
    - filepath: str, path to the input CSV file.
    
    Returns:
    - DataFrame: The cleaned DataFrame without the last column.
    """
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(filepath, index_col='Time' , sep=',', parse_dates=True, low_memory=False)
        
        return df
    
    except FileNotFoundError:
        print(f"Error: The file at {filepath} was not found.")
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
    except pd.errors.ParserError:
        print("Error: There was a problem parsing the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example usage
filepath = r'data/visitor_count/aggregated_historic_visitor_count.csv'
df = load_and_clean_visitor_count_data(filepath)


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from pycaret.time_series import *

# Assuming 'df' is the dataframe with time as index and 95 sensors as columns (hourly frequency)
# Ensure index is DateTime and frequency is set correctly
df.index = pd.to_datetime(df.index)

# Remove duplicates in the index
df = df[~df.index.duplicated(keep='last')]

# Set the frequency to hourly, filling in any missing timestamps
df = df.asfreq('H')

# Impute missing values with forward fill, and use backward fill as a fallback
df = df.fillna(method='ffill').fillna(method='bfill')

# List to store forecast results for each sensor
forecasts = {}

# Loop through each sensor (column) to run the PyCaret model for the next year
for sensor in df.columns:
    print(f"Running forecast for {sensor}")
    
    # Setup PyCaret for the current sensor's data
    exp = setup(data=df[[sensor]], target=sensor, session_id=123, fold=3, fh=8760,  # Forecast horizon for 8760 hours (1 year)
                seasonal_period='H', verbose=False)
    
    # Compare models to find the best for this sensor
    best_model = compare_models()
    
    # Create the model for the sensor
    model = create_model(best_model)
    
    # Forecast the next 8760 hours (1 year) for this sensor
    forecast = predict_model(model, fh=8760)
    
    # Store the forecasted values
    forecast_index = pd.date_range(start=df.index[-1], periods=8761, freq='H')[1:]  # Generating 8760 hours from the last index
    forecast_series = pd.Series(forecast['Label'].values, index=forecast_index)
    forecasts[sensor] = forecast_series
    
    # Plot actual vs forecast for the sensor (last 100 data points and future 8760 points)
    plt.figure(figsize=(12, 6))
    plt.plot(df.index[-100:], df[sensor].tail(100), label='Actual', color='blue')
    plt.plot(forecast_series.index, forecast_series, label='Forecast (Next Year)', color='red')
    plt.title(f'Visitor Count Forecast for {sensor} - Next Year')
    plt.xlabel('Date')
    plt.ylabel('Number of Visitors')
    plt.legend()
    plt.grid(True)
    plt.show()


Running forecast for Bayerisch Eisenstein IN


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:20:16
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,ARIMA


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
naive,Naive Forecaster,14401922096050856.0000,290747320.7314,3.1979,4.3325,12048671331855586.0000,0.3489,0.2678,7.2033
grand_means,Grand Means Forecaster,14401922096050856.0000,290747320.7314,3.1979,4.3325,12048671331855586.0000,0.3489,0.2678,4.5633
snaive,Seasonal Naive Forecaster,14401922096050856.0000,290747320.7314,3.1979,4.3325,12048671331855586.0000,0.3489,0.2678,0.2967
polytrend,Polynomial Trend Forecaster,14401922096051256.0000,290747320.7314,3.1979,4.3325,12048671331855626.0000,0.3489,-0.3988,0.0933


Processing:   0%|          | 0/121 [00:00<?, ?it/s]

KeyboardInterrupt: 